# CompuOpti

In [28]:
import os
import json

import numpy as np
import gurobipy as grb
from gurobipy import GRB

In [41]:
INF = 2 ** 64

18446744073709551616


In [29]:
model = grb.Model()

In [30]:
instance = "toy_instance.json"

file = os.path.join("instances", instance)
data = json.load(open(file, "r"))

## Modeling

### Parameters of the problem

Define the length of indices

In [31]:
# Number of workers
worker_length = len(data["staff"])

# Number of jobs
job_length = len(data["jobs"])

# Number of skills
skill_length = len(data["qualifications"])

# Number of days
day_length = data["horizon"]

Define jobs parameters

In [32]:
gains_job = np.array([job["gain"] for job in data["jobs"]])
penalties_job = np.array([job["daily_penalty"] for job in data["jobs"]])
due_dates_job = np.array([job["due_date"] for job in data["jobs"]])
work_days_job_skill = np.array([
    [
        job["working_days_per_qualification"][skill] if skill in job["working_days_per_qualification"] else 0
        for skill in data["qualifications"]
    ]
    for job in data["jobs"]
])

Define staff parameters

In [33]:
qualifications_worker_skill = np.array([
    [
        1 if skill in worker["qualifications"] else 0
        for skill in data["qualifications"]
    ]
    for worker in data["staff"]
])
vacations_worker_day = np.array([
    [
        1 if 1 + day in worker["vacations"] else 0
        for day in range(day_length)
    ]
    for worker in data["staff"]
])

### Decision variables

Variable to model the full solution

In [34]:
# 4-D array of binary variables : 1 if a worker is assigned to a certain project for a certain skill on a certain day, else 0
works_worker_job_skill_day = model.addVars(worker_length, job_length, skill_length, day_length, vtype=GRB.BINARY, name="work")

Variables to compute the duration of a job

In [36]:
started_after_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="started_after") # 1 if a job is started after a certain day, else 0
finished_before_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="finished_before") # 1 if a job is finished before a certain day, else 0

Variables to compute the total gain

In [35]:
realized_job = model.addVars(job_length, vtype=GRB.BINARY, name="realized") # 1 if a job is realized, else 0
finished_late_job = model.addVars(job_length, vtype=GRB.BINARY, name="finished_late") # 1 if a job is finished late, else 0

Variable to compute the maximum duration per job

In [37]:
max_duration = model.addVar(vtype=GRB.INTEGER, name="max_duration") # Integer that represents the maximum duration for any job

Variables to compute the maximum assignements per worker

In [38]:
assigned_worker_job = model.addVars(worker_length, job_length, vtype=GRB.BINARY, name="assigned") # 1 if a certain worker is assigned on a certain job, else 0
nb_assigned_worker = model.addVars(worker_length, vtype=GRB.INTEGER, name="nb_assigned") # Integers that represents the number of assigned jobs for each worker
max_assigned = model.addVar(vtype=GRB.INTEGER, name="max_assigned") # Integer that represents the maximum number of assigned jobs for any worker

### Constraints

#### Time Table Constraints

Define the constraints of the planning problem itself.

- Worker qualification constraint

In [ ]:
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= qualifications_worker_skill[worker, skill]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="qualification"
)

- Vacation constraint

In [ ]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for job in range(job_length) for skill in range(skill_length)) \
            <= 1 - vacations_worker_day[worker, day]
        for worker in range(worker_length)
        for day in range(day_length)
    ),
    name="vacation"
)

- Uniqueness of the daily assignement : *done with the vacation constraint*

- Job coverage constraint

In [ ]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for worker in range(worker_length) for day in range(day_length)) \
            == realized_job[job] * work_days_job_skill[job, skill]
        for job in range(job_length)
        for skill in range(skill_length)
    ),
    name="job_coverage"
)

- Uniqueness of the realized projects : *done with realized_job either 0 or 1*

#### Variable Constraints

Define the constraints that are related to the definition of additional variables.

- Started after / Finished before

*Example of what is expected :* 

| Day             | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
|-----------------|---|---|---|---|---|---|---|---|
| Working day     | 0 | 0 | 1 | 1 | 1 | 0 | 0 | 0 |
| Started after   | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 1 |
| Finished before | 0 | 0 | 0 | 0 | 0 | 1 | 1 | 1 |

In [ ]:
# started_after == 0 => works == 0
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= started_after_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="started_after"
)

# increasing sequence
model.addConstrs(
    (
        started_after_job_day[job, day] <= started_after_job_day[job, day + 1]
        for job in range(job_length)
        for day in range(day_length - 1)
    ),
    name="started_after_increasing"
)

In [ ]:
# finished before == 1 => works == 0
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= 1 - finished_before_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="finished_before"
)

# increasing sequence
model.addConstrs(
    (
        finished_before_job_day[job, day] <= finished_before_job_day[job, day + 1]
        for job in range(job_length)
        for day in range(day_length - 1)
    ),
    name="finished_before_increasing"
)

- Realized jobs : *done with job coverage constraint*

- Finished late

In [ ]:
# finished_late == 0 => day_length - sum finished_before <= due_date
model.addConstrs(
    (
        day_length - grb.quicksum(finished_before_job_day[job, day] for day in range(day_length)) \
            <= due_dates_job[job] + INF * finished_late_job[job]
        for job in range(job_length)
    ),
    name="finished_late_0"
)

# finished_late == 1 => day_length - sum finished_before >= due_date + 1
model.addConstrs(
    (
        day_length - grb.quicksum(finished_before_job_day[job, day] for day in range(day_length)) \
            >= due_dates_job[job] + 1 - INF * (1 - finished_late_job[job])
        for job in range(job_length)
    ),
    name="finished_late_1"
)

- Maximum duration

In [ ]:
model.addConstrs(
    (
        grb.quicksum(started_after_job_day[job, day] - finished_before_job_day[job, day] for day in range(day_length)) \
            <= max_duration
        for job in range(job_length)
    ),
    name="max_duration"
)

- Assignements

In [ ]:
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= assigned_worker_job[worker, job]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="assigned_worker_job"
)
model.addConstrs(
    (
        grb.quicksum(assigned_worker_job[worker, job] for job in range(job_length)) <= \
            nb_assigned_worker[worker]
        for worker in range(worker_length)
    ),
    name="nb_assigned_worker"
)
model.addConstrs(
    (
        nb_assigned_worker[worker] <= max_assigned
        for worker in range(worker_length)
    ),
    name="max_assigned"
)

### Objectives